In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
# Import packages that will be used
import c2qa
import custom_gates
import numpy as np
import matplotlib.pyplot as plt
from qiskit import ClassicalRegister, visualization, AncillaRegister,QuantumRegister
from qiskit_aer.noise import NoiseModel, amplitude_damping_error
from qiskit.quantum_info import state_fidelity, Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info.operators.predicates import is_unitary_matrix
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.library.standard_gates import HGate
from math import pi, ceil
import scipy
from c2qa.operators import CVOperators
from c2qa.qumoderegister import QumodeRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from c2qa.parameterized_unitary_gate import ParameterizedUnitaryGate
from qutip import *

/home/smohapa5


In [2]:
def qproj00():
    return basis(2, 0).proj()


def qproj11():
    return basis(2, 1).proj()


def qproj01():
    op = np.array([[0, 1], [0, 0]])
    return Qobj(op)


def qproj10():
    op = np.array([[0, 0], [1, 0]])
    return Qobj(op)

In [22]:
def plot_wigner(state, x_range=5, y_range=5, resolution=200):
    """
    Plots the Wigner function (quasiprobability distribution) of a given QuTiP state.

    Parameters:
        state (Qobj): A QuTiP quantum state (density matrix or ket).
        x_range (float): Range for the x-axis (q quadrature).
        y_range (float): Range for the y-axis (p quadrature).
        resolution (int): Number of points for discretization.

    Returns:
        None (Displays the Wigner function plot).
    """
    # Define phase-space grid
    x = np.linspace(-x_range, x_range, resolution)
    y = np.linspace(-y_range, y_range, resolution)
    X, Y = np.meshgrid(x, y)

    # Compute the Wigner function
    W = wigner(state, x, y)

    # Plot the Wigner function
    plt.figure(figsize=(6, 5))
    plt.contourf(X, Y, W, 100, cmap="RdBu_r")  # Red-Blue colormap
    plt.colorbar(label="Wigner function")
    plt.xlabel("q (position quadrature)")
    plt.ylabel("p (momentum quadrature)")
    plt.title("Wigner Distribution")
    plt.grid(True)
    plt.show()

## Elementary Operations

In [6]:
def Q_displacement_plus1(cutoff):
    return (-1j*momentum(cutoff)).expm()
def Q_displacement_minus1(cutoff):
    return (1j*momentum(cutoff)).expm()
def bosonic_sum(cutoff1,cutoff2):
    return (-1j*tensor(position(cutoff1),momentum(cutoff2))).expm()
def single_mode_squeeze(cutoff,squeeze_param):
    return (1j*squeeze_param*(position(cutoff)*momentum(cutoff) + momentum(cutoff)*position(cutoff))/2).expm()
def Q_control_plus1(cutoff):
    return tensor(qproj00(),qeye(cutoff)) + tensor(qproj11(),(-1j*momentum(cutoff)).expm())
def Q_control_minus1(cutoff):
    return tensor(qproj00(),qeye(cutoff)) + tensor(qproj11(),(1j*momentum(cutoff)).expm())
def P_displacement_pi(cutoff,sign):
    return tensor(qproj00(),qeye(cutoff)) + tensor(qproj11(),(1j*np.pi*sign*position(cutoff)).expm())
def rotation_control(cutoff,sign):
    return tensor(qproj00(),qeye(cutoff)) + tensor(qproj11(),(1j*np.pi/2*sign*num(cutoff)).expm())


## Test Elementary Operations

In [32]:
N = 50
q = position(N)

## Q displacement by 1

# system1 = basis(N,0)
# system1_after = Q_displacement_plus1(N) * system1

# print(expect(q, system1))
# print(expect(q, system1_after))

## bosonic sum

# system1 = basis(N,0)
# system2 = displace(N,2/np.sqrt(2)) * basis(N,0)
# print("***Initial State***")
# print(expect(q, system1))
# print(expect(q, system2))
# sum_test = tensor(system2,system1)
# sum_test_after = bosonic_sum(N,N)*sum_test

# system1_after = ptrace(sum_test_after, 1)  # Partial trace over subsystem 2
# print("***Final State***")
# print(expect(q, system1_after))

# system2_after = ptrace(sum_test_after, 0)  # Partial trace over subsystem 1
# print(expect(q, system2_after))

## Single mode squeezing
# system1 = basis(N,0)
# system1_after = single_mode_squeeze(N,1.0) * system1

# plot_wigner(system1_after)

## Controlled Q displacement

system1 = basis(2,0)
system2 = displace(N,2/np.sqrt(2)) * basis(N,0)
print("***Initial State***")
print(expect(sigmaz(), system1))
print(expect(q, system2))
sum_test = tensor(system1,system2)
sum_test_after = Q_control_minus1(N)*sum_test

system1_after = ptrace(sum_test_after, 0)  # Partial trace over subsystem 2
print("***Final State***")
print(expect(sigmaz(), system1_after))

system2_after = ptrace(sum_test_after, 1)  # Partial trace over subsystem 1
print(expect(q, system2_after))



***Initial State***
1.0
2.0000000000000018
***Final State***
1.0000000000000007
2.0000000000000018


In [17]:
test1 = basis(2,0).proj()
test2 = basis(2,1).proj()
test3 = tensor(test1,test2)
print(test3)

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
